In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Hilfsfunktionen
Im folgenden sollen Funktionen implementiert werden, die an verschiedenen Stellen in der Entwicklung des Mühlespiels genutzt werden können. Dazu sind sie so einfach wie möglich gehalten.

`count_player_pieces()` zählt die Steine, die ein anzugebender Spieler gerade auf dem Spielfeld hat. Dafür wird über alle möglichen Positionen iteriert und gezählt, bei wie vielen Positionen der Wert der Position mit dem Wert des Spielers übereinstimmt.

In [ ]:
def count_player_pieces(board, player):
    return [position for ring in board for position in ring].count(player)

`empty_positions()` gibt eine Liste mit Tupeln zurück. Jedes Tupel der Form `(ring, cell)` beschreibt eine Stelle auf dem Spielfeld, auf der kein Spielstein steht, also der Wert 0 ist. So erhält man alle leeren Positionen.

In [ ]:
def empty_positions(board):
    return player_pieces(board, 0)

`neighboring_positions()` liefert für eine gegebene Position auf dem Spielfeld alle benachbarten Felder. Die Position wird dabei in der der Form `(ring, cell)` übergeben. Je nachdem wo sich die übergebene Stelle befindet, hat sie 2, 3 oder 4 Nachbarfelder, die es zu ermitteln gilt. Zunächst werden der linke und der rechte Nachbar ermittelt, den jede Position besitzt. Da die Zellen von 0 bis 7 durchnummeriert sind, muss 1 zu dem Zellenwert dazu addiert oder abgezogen werden. Um von 0 zu 7 und zurück zu gelangen, wird das Ergebnis noch Modulo 8 genommen.
Die Zellen 1, 3, 5, 7 liegen in der Mitte und haben deshalb noch einen weiteren Nachbar, falls sie in Ring 0 oder 2 sind. Ansonsten haben sie sogar noch zwei weitere Nachbarpositionen.

In [ ]:
def neighboring_positions(position):
    ring, cell = position
    
    left_neighbor  = (ring, (cell - 1) % 8)
    right_neighbor = (ring, (cell + 1) % 8)
        
    positions = [left_neighbor, right_neighbor]
        
    if cell % 2 == 1:
        if ring in [0, 2] :
            positions.append((1, cell))
        else:
            positions.append((0, cell))
            positions.append((2, cell))
    return positions

`empty_neighbors()` berechnet ausgehend von einem gewählten Stein alle benachbarten leeren Felder.
Hierfür wird ein Spielbrett, sowie die Position des ausgewählten Steines mit ring und cell übergeben. Zurückgegeben wird eine Liste der Position.

In [ ]:
def empty_neighbors(board, ring, cell):
    neighboring = set(neighboring_positions((ring, cell)))
    empty = set(empty_positions(board))
    return neighboring.intersection(empty)

`player_pieces()` gibt eine Liste mit allen Positionen eines zu übergebenden Spielers zurück. Jeder Eintrag der Form `(ring, cell)` beschreibt eine Stelle auf dem Spielfeld, auf der ein Spielstein des Spielers steht, also der Wert gleich dem Wert des Spielers ist.

In [ ]:
def player_pieces(board, player):
    return [(ring, cell) for ring in range(3) for cell in range(8) if board[ring][cell] == player]

`opponent()` gibt den Gegner des übergebenen Spielers zurück. Falls 1 übergeben wird, kommt 2 zurück, ansonsten andersrum.

In [ ]:
def opponent(player):
    return 3 - player

Die Funktion `to_tuple()` wandelt einen übergebenen Spielstatus in Tupelform um.

In [ ]:
def to_tuple(state):
    return (tuple(state[0]), tuple(tuple(ring) for ring in state[1]))

Die Funktion `to_list()` wandelt einen übergebenen Spielstatus in Listenform um.

In [ ]:
def to_list(state):
    return [list(state[0]), [list(ring) for ring in state[1]]]

## Mühlen
`find_mills()` gibt alle Mühlen zurück, die ein Spieler auf dem übergebenen Spielbrett hat. Hierzu werden in der ersten Schleife alle Mühlen ermittelt, die sich auf einem einzigen Ringen befinden. Die zweite Schleife erkennt alle Mühlen, die sich über alle drei Ringe erstrecken. Die Eigenschaft einer Mühle ist, dass drei Steine in gerader Linie direkt nebeneinander liegen. Mühlen werden dabei als Tupel der Form `((ring 1, cell 1), (ring 2, cell 2), (ring 3, cell 3))` gespeichert.

In [ ]:
def find_mills(board, player):
    mills = set()

    for ring in range (0,3):
        for cell in [0,2,4,6]:
            if(board[ring][cell] == board[ring][cell + 1] == board[ring][(cell + 2) % 8] == player):
                mills.add(((ring, cell), (ring, cell + 1), (ring, (cell + 2) % 8)))
    for cell in [1,3,5,7]:
        if(board[0][cell] == board[1][cell] == board[2][cell] == player):
            mills.add(((0, cell), (1, cell), (2, cell)))            
    return mills

`count_new_mills()` nimmt zwei Mengen von Mühlen und berechnet die Anzahl der Unterschiede zwischen `old_mills` und `new_mills`.

In [ ]:
def count_new_mills(old_mills, new_mills):
    return len(new_mills.difference(old_mills))

`get_opponent_beatable_pieces()` gibt eine Liste mit entfernbaren Steinen des Gegners zurück. Dies ist notwendig, wenn auf Grund des Bildens einer Mühle Steine des Gegners entfernt werden dürfen.
Dazu werden alle Steine sowie alle Mühlsteine des Gegners ermittelt. Die schlagbaren Steine sind die Steine des Spielers ohne dessen Mühlsteine. Ist das Ergebnis jedoch leer, werden alle Steine des Gegners zurückgegeben, da laut den offiziellen Regeln in diesem Fall auch Steine aus Mühlen entfernt werden können. Die Arbeit mit Mengen bringt hier keinen Performancevorteil.

In [ ]:
def get_opponent_beatable_pieces(board, current_player):
    opponent_player = opponent(current_player)

    opponent_mill_pieces = [(ring, cell) for mill in find_mills(board, opponent_player) for (ring, cell) in mill]
    opponent_pieces      = player_pieces(board, opponent_player)
    beatable_pieces      = [piece for piece in opponent_pieces if piece not in opponent_mill_pieces]
    
    return beatable_pieces if len(beatable_pieces) != 0 else opponent_pieces

`beat_pieces()` gibt alle möglichen Spielfelder zurück, nachdem eine bestimmte Anzahl an schlagbaren gegnerischen Steinen (`count`) entfernt, also durch den Wert 0 ersetzt, wurde. Die Eingabeparameter sind dazu ein Spielbrett, also der zweite Teile von `state`, eine natürliche Zahl, die der Anzahl der zu entfernenden Steine entspricht, und ein Spieler, der die Steine entfernt. Die Methode funktioniert rekursiv. Als Rekursionsanker dient `count <= 0`. Das Dupliket entfernen wäre schöner, wenn man statt den Listen Mengen verwenden würde. Dazu müsste aber State auf Tupel umgestellt werden, da Lists mutable sind. Wenn noch umgeformt werden muss, bringt das keinen Performancevorteil.

In [ ]:
def beat_pieces(board, count, player):
    if count <= 0: return [board]
    
    boards = []
    beatable_pieces = get_opponent_beatable_pieces(board, player)
    for ring, cell in beatable_pieces:
        new_board = copy.deepcopy(board)
        new_board[ring][cell] = 0
        boards.extend(beat_pieces(new_board, count - 1, player))
    
    # Duplikate entfernen
    result = []
    for board in boards:
        if board not in result:
            result.append(board)
    return result